In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split



In [2]:
data = pd.read_csv('data/data_fusion.csv')
data = data.dropna()
data.head()

,Titre,Description,Auteurs,Image,Lien Google,Editeur,Date publication,infoLink,Genre,Nb scores,Id,Prix,User_id,Nom lecteur,revue/utilité,revue/score,revue/heure,revue/résumé,revue/texte
47,The Church of Christ: A Biblical Ecclesiology ...,In The Church of Christ: A Biblical Ecclesiolo...,['Everett Ferguson'],http://books.google.com/books/content?id=kVqRa...,http://books.google.nl/books?id=kVqRaiPlx88C&p...,Wm. B. Eerdmans Publishing,1996,http://books.google.nl/books?id=kVqRaiPlx88C&d...,['Religion'],5.0,0802841899,25.97,ARI272XF8TOL4,Christopher J. Bray,74/81,5.0,955411200,Ecclesiological Milestone,With the publication of Everett Ferguson's boo...
48,The Church of Christ: A Biblical Ecclesiology ...,In The Church of Christ: A Biblical Ecclesiolo...,['Everett Ferguson'],http://books.google.com/books/content?id=kVqRa...,http://books.google.nl/books?id=kVqRaiPlx88C&p...,Wm. B. Eerdmans Publishing,1996,http://books.google.nl/books?id=kVqRaiPlx88C&d...,['Religion'],5.0,0802841899,25.97,A36TPZSH8LBT1,haskell,2/3,5.0,1311465600,Early Christian development of the Church,Everett Ferguson approaches the subject of ear...
49,The Church of Christ: A Biblical Ecclesiology ...,In The Church of Christ: A Biblical Ecclesiolo...,['Everett Ferguson'],http://books.google.com/books/content?id=kVqRa...,http://books.google.nl/books?id=kVqRaiPlx88C&p...,Wm. B. Eerdmans Publishing,1996,http://books.google.nl/books?id=kVqRaiPlx88C&d...,['Religion'],5.0,0802841899,25.97,ANX3DDV12ZRRU,GodsBreath.wordpress,2/3,4.0,1289952000,An Excellent Presentation of the Beliefs of th...,This book is a continual resource. It is so bi...
50,The Church of Christ: A Biblical Ecclesiology ...,In The Church of Christ: A Biblical Ecclesiolo...,['Everett Ferguson'],http://books.google.com/books/content?id=kVqRa...,http://books.google.nl/books?id=kVqRaiPlx88C&p...,Wm. B. Eerdmans Publishing,1996,http://books.google.nl/books?id=kVqRaiPlx88C&d...,['Religion'],5.0,0802841899,25.97,A2H2LORTA5EZY2,Edward E. Howe,3/5,4.0,1266192000,Christ is Lord,This is a very useful and thorough text book. ...
195,The Battleship Bismarck,The Bismarck is perhaps the most famous – and ...,['Stefan Draminski'],http://books.google.com/books/content?id=nxttD...,http://books.google.nl/books?id=nxttDwAAQBAJ&p...,Bloomsbury Publishing,20/09/2018,https://play.google.com/store/books/details?id...,['History'],1.0,0887402216,34.95,A30IUAABSEHEPI,Lawrence Duckles,9/9,3.0,1055980800,The Battleship Bismarck reviewed,This book is both a history and a photo album ...


In [3]:
from sklearn.model_selection import KFold

X_polarite = data.drop(['Image', 'Lien Google', 'infoLink', 'Id', 'revue/score'], axis=1)
Y_polarite = data['revue/score']

X_train_polarite, X_test_polarite, Y_train_polarite, Y_test_polarite = train_test_split(X_polarite, Y_polarite, test_size=0.2, random_state=42)

In [13]:
from sklearn.preprocessing import OneHotEncoder, RobustScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from scipy.sparse import hstack
import pandas as pd

# Identifier les types de colonnes
colonnes_categoriques = ['Auteurs', 'Editeur', 'Genre', 'Nom lecteur', 'revue/utilité']
colonnes_numeriques = ['Nb scores', 'Prix', 'revue/heure']
colonnes_textuelles = ['Titre', 'revue/résumé', 'revue/texte']

# Transformer les colonnes catégorielles et numériques
preprocessor = ColumnTransformer([
    ('cat', OneHotEncoder(handle_unknown='ignore'), colonnes_categoriques),
    ('num', RobustScaler(), colonnes_numeriques),
])

# Appliquer fit_transform sur X_train, mais SEULEMENT transform sur X_test
X_train_cat_num = preprocessor.fit_transform(X_train_polarite)
X_test_cat_num = preprocessor.transform(X_test_polarite)

# Appliquer TF-IDF sur chaque colonne textuelle séparément
text_pipelines = {col: TfidfVectorizer(max_features=500) for col in colonnes_textuelles}

X_train_text = [text_pipelines[col].fit_transform(X_train_polarite[col].fillna("").astype(str)) for col in colonnes_textuelles]
X_test_text = [text_pipelines[col].transform(X_test_polarite[col].fillna("").astype(str)) for col in colonnes_textuelles]

# Concaténer les matrices obtenues pour chaque colonne textuelle
X_train_text_combined = hstack(X_train_text)
X_test_text_combined = hstack(X_test_text)

# Vérification des dimensions après transformation
print(f"Shape X_train_cat_num: {X_train_cat_num.shape}")
print(f"Shape X_test_cat_num: {X_test_cat_num.shape}")
print(f"Shape X_train_text_combined: {X_train_text_combined.shape}")
print(f"Shape X_test_text_combined: {X_test_text_combined.shape}")

# Concaténer toutes les matrices ensemble
if X_train_cat_num.shape[0] == X_train_text_combined.shape[0] and X_test_cat_num.shape[0] == X_test_text_combined.shape[0]:
    X_train_encoded = hstack([X_train_cat_num, X_train_text_combined])
    X_test_encoded = hstack([X_test_cat_num, X_test_text_combined])

    # Vérification finale des dimensions
    print("✅ Shape X_train_encoded:", X_train_encoded.shape)
    print("✅ Shape X_test_encoded:", X_test_encoded.shape)
else:
    print("❌ Problème de dimensions entre les colonnes non-textuelles et textuelles.")


Shape X_train_cat_num: (139436, 119165)
Shape X_test_cat_num: (34859, 119165)
Shape X_train_text_combined: (139436, 1500)
Shape X_test_text_combined: (34859, 1500)
✅ Shape X_train_encoded: (139436, 120665)
✅ Shape X_test_encoded: (34859, 120665)


In [15]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error

# Initialisation du modèle Gradient Boosting
gb_model = GradientBoostingRegressor(n_estimators=500, learning_rate=0.05, max_depth=6, random_state=42)

# Entraînement
gb_model.fit(X_train_encoded, Y_train_polarite)

# Prédiction
Y_pred_gb = gb_model.predict(X_test_encoded)

# Évaluation
mse_gb = mean_squared_error(Y_test_polarite, Y_pred_gb)
print(f"📊 MSE avec Gradient Boosting : {mse_gb:.4f}")


KeyboardInterrupt: 